[kaggle_url](https://www.kaggle.com/code/linakeepgoing/2-etfs/notebook)

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("linakeepgoing/etfs-main")

# print("Path to dataset files:", path)

`A 트레이더` : 주가의 움직임을 중시하며, 이동평균, RSI와 같은 기술 지표를 활용.  
`B 트레이더` : 소비자 물가, 금리 등 거시경제 지표를 기반으로 투자 의사결정.  
`C 트레이더` : 금, 달러 등 다양한 자산 간의 상관관계에 초점을 맞추어 트레이딩.

> ETFs 데이터를 설명 변수로 사용하는 이유
1. ETF는 다양한 자산을 포함하는 펀드로서, 단일 주식이나 자산에 비해 시장의 광범위한 동향을 더 잘 포착한다. 이는 개별 주식의 특정 리스크를 줄이고, 전체 시장 또는 특정 섹터의 평균적인 성과를 추적하는 데 유용.
2. 거시경제 지표 대신 ETF 데이터를 선택하는 주된 이유는 실시간성과 접근성. 거시경제 지표는 발표가 지연되거나 때때로 수정될 수 있는 반면, ETF는 거래소에서 실시간으로 거래되어 시장의 현재 상태를 더 정확히 반영. 

In [ ]:
# 1. Librarys
import warnings
import glob
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn import svm
import seaborn as sns; sns.set_theme()

In [ ]:
# 2. ETFs_main.csv 데이터 로드
df = pd.read_csv('./data/ETFs_main.csv')
df.head(3)

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO
0,2007-02-20,146.04,145.56,146.20,144.0,56909500.0,65.31,83.51,2.3263,0.31,48.67,25.07,10.24,40.055
1,2007-02-21,145.98,145.61,146.07,145.0,63971500.0,67.28,82.90,2.3653,0.32,49.86,25.12,10.20,39.975
2,2007-02-22,145.87,146.05,146.42,145.0,79067398.0,67.15,82.46,2.3871,0.31,50.33,25.12,10.18,40.220


In [ ]:
# 3. 기술적 지표 만들기
def moving_average(df, n):
    MA = pd.Series(df['CLOSE_SPY'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))
    df = df.join(MA)
    return df

def volume_moving_average(df, n):
    VMA = pd.Series(df['VOLUME'].rolling(n, min_periods=n).mean(), name='VMA_' + str(n))
    df = df.join(VMA)
    return df

def relative_strength_index(df, n):
    """
    RSI 계산
    0 < RSI < 100 
    RSI > 70 : 과매수 상태
    RSI < 30 : 과매도 상태 
    """
    delta = df['CLOSE_SPY'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=n).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=n).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    RSI.name = 'RSI_' + str(n)
    df = df.join(RSI)
    return df

In [ ]:
# 4. 기술 지표 적용
df = moving_average(df, 45) # 실제 경과 일수는 60일이지만, 영업일 기준인 45일로 작성
df = volume_moving_average(df, 45)
df = relative_strength_index(df, 14) # 시장강도 지수는 보통 14일이나 21일 사용 

# 'Dates' 열을 인덱스로 설정
df = df.set_index('Dates')
df = df.dropna()
print(len(df))

2727


In [16]:
# 5. target 변수  

# 타겟 변수 생성 (pct_change)
df['pct_change'] = df['CLOSE_SPY'].pct_change()

# 모델링을 위한 이진 분류 값 생성
df['target'] = np.where(df['pct_change'] > 0, 1, 0)
df = df.dropna(subset=['target'])  # 결측값 제거

# 정수형 변환
df['target'] = df['target'].astype(np.int64)

print(df['target'].value_counts())

target
1    1471
0    1256
Name: count, dtype: int64


In [17]:
# 6. 다음날 예측을 위해 타겟 변수를 shift
df['target'] = df['target'].shift(-1)
df = df.dropna()
print(len(df))

2725


In [18]:
# 7. 설명 변수와 타겟 변수 분리
y_var = df['target']
x_var = df.drop(['target', 'OPEN', 'HIGH', 'LOW', 'VOLUME', 'CLOSE_SPY', 'pct_change'], axis=1)

In [19]:
# 8. 상승과 하락 비율 확인
up = df[df['target'] == 1].target.count()
total = df.target.count()
print('up/down ratio: {0:.2f}'.format(up / total))

up/down ratio: 0.54


In [20]:
# 9. 훈련셋과 테스트셋 분할
X_train, X_test, y_train, y_test = train_test_split(x_var, y_var, test_size=0.3, shuffle=False, random_state=3)

# 훈련셋과 테스트셋의 양성 샘플 비율 확인
train_count = y_train.count()
test_count = y_test.count()

print('train set label ratio')
print(y_train.value_counts() / train_count)
print('test set label ratio')
print(y_test.value_counts() / test_count)

train set label ratio
target
1.0    0.543786
0.0    0.456214
Name: count, dtype: float64
test set label ratio
target
1.0    0.530562
0.0    0.469438
Name: count, dtype: float64


In [21]:
x_var.head(3 )

,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14
Dates,,,,,,,,,,,
2007-04-30,67.09,83.7166,2.4361,0.57,51.24,24.49,14.22,40.935,143.601556,1.116466e+08,70.956720
2007-05-02,66.66,83.3800,2.4366,0.59,49.59,24.66,13.08,42.020,143.680667,1.121613e+08,79.237288
2007-05-03,67.49,83.1100,2.4346,0.60,49.28,24.69,13.09,42.435,143.780222,1.123421e+08,79.604579


In [22]:
# 10. 혼동 행렬 및 성능 평가 함수
def get_confusion_matrix(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('confusion matrix')
    print(confusion)
    print('accuracy: {0:.4f}, precision: {1:.4f}, recall: {2:.4f}, F1: {3:.4f}, ROC AUC score: {4:.4f}'.format(
        accuracy, precision, recall, f1, roc_score))

In [23]:
# 11. 모델 학습 및 평가
# XGBoost 모델 학습 및 예측
xgb_dis = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_dis.fit(X_train, y_train)
xgb_pred = xgb_dis.predict(X_test)

# 훈련 정확도 확인
print(xgb_dis.score(X_train, y_train))

# 성능 평가
get_confusion_matrix(y_test, xgb_pred)

0.8479286837965391
confusion matrix
[[333  51]
 [358  76]]
accuracy: 0.5000, precision: 0.5984, recall: 0.1751, F1: 0.2709, ROC AUC score: 0.5212


In [ ]:
# 12. 랜덤 포레스트 매개변수 설정
n_estimators = range(10, 200, 10)
params = {
    'bootstrap': [True],
    'n_estimators': n_estimators,
    'max_depth': [4, 6, 8, 10, 12],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [2, 4, 6, 8, 10],
    'max_features': [4]
}

# 교차 검증 설정
my_cv = TimeSeriesSplit(n_splits=5).split(X_train)

# GridSearchCV를 사용한 모델 학습
clf = GridSearchCV(RandomForestClassifier(), params, cv=my_cv, n_jobs=-1)
clf.fit(X_train, y_train)

# 최적의 매개변수와 정확도 출력
print('best parameter:\n', clf.best_params_)
print('best prediction: {0:.4f}'.format(clf.best_score_))

In [ ]:
# 13. 테스트셋에서의 성능 확인
pred_con = clf.predict(X_test)
accuracy_con = accuracy_score(y_test, pred_con)
print('accuracy: {0:.4f}'.format(accuracy_con))
get_confusion_matrix(y_test, pred_con)

In [ ]:
# 14. 타겟 변수 통계 확인
df['pct_change'].describe()